### scripts

<a href="https://colab.research.google.com/github/hzwer/arXiv2020-RIFE/blob/main/Colab_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/hzwer/arXiv2020-RIFE

In [ ]:
!mkdir /content/arXiv2020-RIFE/train_log
%cd /content/arXiv2020-RIFE/train_log
!gdown --id 1APIzVeI-4ZZCEuIRE1m6WYfSCaOsi_7_
!7z e RIFE_trained_model_v3.6.zip

In [ ]:
%cd /content/arXiv2020-RIFE/
!gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
!pip3 install -r requirements.txt

Please upload your video to content/arXiv2020-RIFE/video.mp4, or use our demo video.

In [ ]:
!nvidia-smi
!python3 inference_video.py --exp=2 --video=demo.mp4 --montage --skip

Our demo.mp4 is 25FPS. You can adjust the parameters for your own perference.
For example: 
--fps=60 --exp=1 --video=mydemo.avi --png

In [ ]:
from IPython.display import display, Image
import moviepy.editor as mpy
display(mpy.ipython_display('demo_4X_100fps.mp4', height=256, max_duration=100.))

In [ ]:
!python3 inference_img.py --img demo/I0_0.png demo/I0_1.png
ffmpeg -r 10 -f image2 -i output/img%d.png -s 448x256 -vf "split[s0][s1];[s0]palettegen=stats_mode=single[p];[s1][p]paletteuse=new=1" output/slomo.gif
# Image interpolation

### as-is

In [68]:
import subprocess

from PIL import Image
from pathlib import Path

from typing import List, Union


VIDS_DIR = Path("vids")


def correct_shape(img, n_w, n_h):
    w, h = img.size
    if w % n_w != 0:
        target_w = w - w % n_w
    if h % n_h != 0:
        target_h = h - h % n_h
    else:
        return img

    print(f"tile vs dim. mismatch, resizing {w, h} -> {target_w, target_h}")
    img = img.resize((target_w, target_h))
    return img


def untile(img_path, res_dir, n_tw=3, n_th=3):
    img = Image.open(img_path)
    img = correct_shape(img, n_tw, n_th)
    w, h = img.size

    dir_to_save = res_dir / f"{img_path.stem}-untiled/"
    dir_to_save.mkdir(exist_ok=True)
    print(f"saving to {dir_to_save}")
    
    tile_w, tile_h = w // n_tw, h // n_th
    for i in range(n_th):
        y = i * tile_h
        for j in range(n_tw):
            x = j * tile_w
            print(f"tile-n{i}m{j} \t x={j} * {tile_w}: {x} \t y = {i} * {tile_h}: {y} \t \n")
            left_upper = (x, y)
            right_bottom = (x + tile_w, y + tile_h)
            print(f"lu: {left_upper}, rb: {right_bottom}")
            cur_img = img.crop((x, y, x + tile_w, y + tile_h))
            cur_img.save(dir_to_save / f"{i}{j}.png")
            

def process_images(
    image_pathes: List[Union[str, Path]], 
    rife_output_dir: Path,
    ims_mask: str,
    vids_dir: Path = VIDS_DIR,
) -> str:
    """
    writes resulting video  path of resulting video to 
    {VIDS_DIR}/{rife_output_dir}.mp4
    """
        
    ims_pair = []
    for i in range(len(image_pathes) - 1):
        ims_pair.append(
            (ims_path[i].as_posix(), ims_path[i + 1].as_posix())
        )
    ims_pair.append(
        (ims_path[-1].as_posix(), ims_path[0].as_posix())
    )
    
    
    save_dir = rife_output_dir / "interpolated" / ims_mask
    save_dir.mkdir(exist_ok=True, parents=True)
    
    print(f"saving interpolated to {save_dir}")
    for im1_path, im2_path in ims_pair:
        start_ind = len([i for i in save_dir.glob("*.png")])
        print(f"starting from ... {start_ind}")
        ! python3 inference_img.py --img {im1_path} {im2_path} --exp=4 --save_dir={save_dir} --start_ind={start_ind}


    VIDS_DIR.mkdir(exist_ok=True)
    
    print(f" saving video to: {VIDS_DIR}/{ims_mask}.mp4")
    ! ffmpeg -v 0 -framerate 60 -i {save_dir}/%05d.png -c:v libx264 -pix_fmt yuv420p {VIDS_DIR}/{rife_output_dir}_{ims_mask}.mp4


In [35]:
ims_dir = Path("./alb-bc/")
result_dir = Path("alb-bc-untiled/")
result_dir.mkdir(exist_ok=True)


nw, nh = 3, 3
for i in ims_dir.glob("*.png"):
    untile(i, result_dir, nw, nh)

saving to alb-bc-untiled/grid-0014-untiled
tile-n0m0 	 x=0 * 512: 0 	 y = 0 * 512: 0 	 

lu: (0, 0), rb: (512, 512)
tile-n0m1 	 x=1 * 512: 512 	 y = 0 * 512: 0 	 

lu: (512, 0), rb: (1024, 512)
tile-n0m2 	 x=2 * 512: 1024 	 y = 0 * 512: 0 	 

lu: (1024, 0), rb: (1536, 512)
tile-n1m0 	 x=0 * 512: 0 	 y = 1 * 512: 512 	 

lu: (0, 512), rb: (512, 1024)
tile-n1m1 	 x=1 * 512: 512 	 y = 1 * 512: 512 	 

lu: (512, 512), rb: (1024, 1024)
tile-n1m2 	 x=2 * 512: 1024 	 y = 1 * 512: 512 	 

lu: (1024, 512), rb: (1536, 1024)
tile-n2m0 	 x=0 * 512: 0 	 y = 2 * 512: 1024 	 

lu: (0, 1024), rb: (512, 1536)
tile-n2m1 	 x=1 * 512: 512 	 y = 2 * 512: 1024 	 

lu: (512, 1024), rb: (1024, 1536)
tile-n2m2 	 x=2 * 512: 1024 	 y = 2 * 512: 1024 	 

lu: (1024, 1024), rb: (1536, 1536)
saving to alb-bc-untiled/grid-0119-untiled
tile-n0m0 	 x=0 * 512: 0 	 y = 0 * 512: 0 	 

lu: (0, 0), rb: (512, 512)
tile-n0m1 	 x=1 * 512: 512 	 y = 0 * 512: 0 	 

lu: (512, 0), rb: (1024, 512)
tile-n0m2 	 x=2 * 512: 1024 	 y = 

In [69]:
ims_mask = [f"{i}{j}" for i in range(nw) for j in range(nh)]

for im_m in ims_mask:
    ims_path = [i for i in result_dir.glob(f"*/{im_m}.png")]
    
    process_images(ims_path, result_dir, im_m)

saving interpolated to alb-bc-untiled/interpolated/00
starting from ... 0
Loaded v3.x HD model.
starting from ... 17
Loaded v3.x HD model.
starting from ... 34
Loaded v3.x HD model.
starting from ... 51
Loaded v3.x HD model.
starting from ... 68
Loaded v3.x HD model.
starting from ... 85
Loaded v3.x HD model.
starting from ... 102
Loaded v3.x HD model.
starting from ... 119
Loaded v3.x HD model.
starting from ... 136
Loaded v3.x HD model.
starting from ... 153
Loaded v3.x HD model.
starting from ... 170
Loaded v3.x HD model.
starting from ... 187
Loaded v3.x HD model.
starting from ... 204
Loaded v3.x HD model.
starting from ... 221
Loaded v3.x HD model.
starting from ... 238
Loaded v3.x HD model.
starting from ... 255
Loaded v3.x HD model.
starting from ... 272
Loaded v3.x HD model.
starting from ... 289
Loaded v3.x HD model.
starting from ... 306
Loaded v3.x HD model.
starting from ... 323
Loaded v3.x HD model.
starting from ... 340
Loaded v3.x HD model.
starting from ... 357
Loaded 

In [72]:
import time
import numpy as np

In [75]:
def time_inf(ims_pair, exp=2):
    times = []
    save_dir = Path("dump")
    save_dir.mkdir(exist_ok=True)
    for im1_path, im2_path in ims_pair:
        start_ind = len([i for i in save_dir.glob("*.png")])
        s_time = time.time()
        ! python3 inference_img.py --img {im1_path} {im2_path} --exp={exp} --save_dir={save_dir} --start_ind={start_ind}
        exec_s = time.time() - s_time
        times.append(exec_s)
        
    return times


dict_times = {}

ims_dir = Path("./samples-abl")
ims_path = [i for i in ims_dir.glob("*.png")]
ims_pair = []

for i in range(len(ims_path) - 1):
    ims_pair.append(
        (ims_path[i].as_posix(), ims_path[i + 1].as_posix())
    )
ims_pair.append(
    (ims_path[-1].as_posix(), ims_path[0].as_posix())
)

dict_times["exp4_512x512"] = time_inf(ims_pair, exp=4)

dict_times["exp2_512x512"] = time_inf(ims_pair, exp=2)


ims_dir = Path("./sample-alb-upscaled/")
ims_path = [i for i in ims_dir.glob("*.png")]
ims_pair = []

for i in range(len(ims_path) - 1):
    ims_pair.append(
        (ims_path[i].as_posix(), ims_path[i + 1].as_posix())
    )
ims_pair.append(
    (ims_path[-1].as_posix(), ims_path[0].as_posix())
)

dict_times["exp4_2048x2048"] = time_inf(ims_pair, exp=4)  


Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
Loaded v3.x HD model.
tag: 	 exp4_512x512
Mean exec. time +- std 	 2.6263 +- 0.0287
Median exec. time +- std 	 2.6237 +- 0.0287
tag: 	 exp2_512x512
Mean exec. time +- std 	 2.4216 +- 0.0161
Median exec. time +- std 	 2.4253 +- 0.0161
tag: 	 exp4_2048x2048
Mean exec. time +- std 	 7.3876 +- 0.0298
Median exec. time +- std 	 7.3966 +- 0.0298


In [76]:
for name in dict_times:
    times = dict_times[name]
    print(f"tag: \t {name}")
    print(f"mean +- std \t {np.mean(times):.4f} +- {np.std(times):.4f}")
    print(f"median +- std \t {np.median(times):.4f} +- {np.std(times):.4f}")
    print("\n\n")

tag: 	 exp4_512x512
mean +- std 	 2.6263 +- 0.0287
median +- std 	 2.6237 +- 0.0287



tag: 	 exp2_512x512
mean +- std 	 2.4216 +- 0.0161
median +- std 	 2.4253 +- 0.0161



tag: 	 exp4_2048x2048
mean +- std 	 7.3876 +- 0.0298
median +- std 	 7.3966 +- 0.0298





In [ ]:
# TODO:
# full load on GPU
# profile (*)
# continuosly stream latest
# 2x -> 4x
# put in .inference [bs, * , * ...] tensor
# fork
# different ratios